# Machine Learning Engineer Nanodegree 
## Supervised Learning
## Project: Building a Student Intervention System

## Author: Mai Aye

In this notebook, I will build a student intervention system by implementing various classification models.  My goal for this project is identify students who might need early intervention before they fail to graduate.  

### 1 - Classification vs. Regression

In order to predict students who will or will not graduate, I will use classification models, because I am predicting **discrete** labels or values of graduating (1) or not graduating (0).  These discrete labels are also known as **class**, and classification models predict whether a record belongs to a particular class.

## Exploring the Data
Exploring the data, the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [105]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students.  I will need compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [106]:
# Calculating number of students
n_students = np.array(student_data["passed"]).shape[0]

# Calculating number of features
features = student_data.drop(["passed"],axis=1)
n_features = len(features.columns)

# Calculating passing students
n_passed = np.count_nonzero(student_data["passed"] == "yes")

# Calculating failing students
n_failed = np.count_nonzero(student_data["passed"] == "no")

# Calculating graduation rate
grad_rate = (float(n_passed)/float(n_students))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, I will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data I obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

I will separate the student data into feature and target columns to see if any features are non-numeric.

In [107]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

There are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [108]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, I will implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [109]:
#Importing any additional functionalities
from sklearn.cross_validation import StratifiedShuffleSplit

#Setting the number of training points
num_train = 300

#Setting the number of testing points
num_test = X_all.shape[0] - num_train

# Doing Shuffle and split the dataset into the number of training and testing points above
#get training and testing index
train_i, test_i = next(iter(StratifiedShuffleSplit(y=y_all, n_iter=1,train_size=num_train,test_size=num_test,random_state=42)
                           ))
X_train = X_all.iloc[train_i]
X_test = X_all.iloc[test_i]
y_train = y_all.iloc[train_i]
y_test = y_all.iloc[test_i]

# Showing the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
                       

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, I will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. I will first discuss the reasoning behind choosing these three models by considering what I know about the data and each model's strengths and weaknesses. I will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. I will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**I will implement the following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **:**
- Decision Trees
- Ensemble Methods (Gradient Boosting)
- Stochastic Gradient Descent (SGD)

### 2 - Model Application

I will be using Decision Tree, Gradient Tree Boosting, and Stochastic Gradient Descent(SGD) to predict student retention. 

**Decision Tree** classifies student retention by learning decision or if-then rules in a tree-like structure deduced from the training data.  The final tree contains *decision nodes* that are features with two or more branches and *leaf nodes* that represent the final classification or decision.  Decision tree works by 1) putting the best feature as decision node at the tree's root, 2) splitting the data into subsets that contain homogenous samples, and repeating steps 1 and 2 until arriving at the final predictions.  Partioning data into decision nodes that contain homogenous instances is determined by *entropy* and *information gain*.  On one hand, *entropy* measures randomness, heterogenuity, or impurity of instances in each decision node.  But on the other hand, *information gain* measures the purity or homogeneity of instances in each decision node, representing the expected amount of information required to predict a new instance.  Hence, the best feature for splitting data into subsequent decision nodes or subsets of similar instances has decreasing entropy and returns the highest information gain.  The *advantages* of Decision Tree are:       
 
  **1. Decision Tree does automatic feature selection**  
       The decision nodes where the tree splits serve as the most important features in predicting class lables.
         
  **2. Decision Tree doesn't require a lot of effort in data preparation**  
       Unlike other classification and regression models that require scaling and normalization, decision tree doesn't need
       a lot of data transformations and can handle missing values and outliers.
       
  **3. Decision Tree can handle non-linear relationships between features**  
       Failing to account for non-linear relationships between features could make models invalid in other models.  However,
       Decision Tree doesn't make any assumptions about relationships between features.  Thus, feature engineering is not   required.  
       
  **4. Decision Tree is easier to explain**  
       Decision trees are easy to explain to non-specialist audience, because we could visualize a set of decisions or rules used to predict student retention.  
       
However, the *disadvantage* of Decision Tree is its tendency to overfit on the training data and then making poor predictions, because overfitting occurs when the model is too complex.  A decision tree's complexity entail the number of decisions or nodes in the tree.  Therefore, we would need to control the number of nodes in order to prevent overfitting.

In our data, we have a lot of features and potentiall non-linear relationships between these features as well as outliers.  As a result, Decision Tree is a good candidate for this problem since it won't require a lot of efforts in data transformations.  Because Decision Tree also generates business rules through decision nodes, we could also use these rules to effectively diagnose and prevent students' failures. 

**Ensemble methods** combine predictions of multiple models  to improve one model's ability to generalize.  **Boosting** in ensemble methods entails building models sequentially and then reducing the *bias* of the combined models.  *Bias*, also known as *underfitting*, involes a model's inability to capture the subtleties or complexities in the training data.  In other words, a biased model is just a model that's too simple.  The goal in boosting then is to incorporate multiple weak models to generate a robust ensemble of models with great predictive powers.  Using the gradient descent method, **Gradient Tree Boosting** first sequentially trians multiple models. Then, each model gradually decreases the loss function designed to minimize prediction errors, fitting new models for accurate predictions and finally providing an ensemble of models that minimize prediction errors.  The advantages of **Gradient Tree Boosting** include being able to handle heterogeneous features, robust to outliers, and result in more accurate predictions.  However, it could be harder to scale and parrallelize due to sequentially building models. **Gradient Tree Boosting** is also a good candidate for this problem because it provides predictive power.

**Gradient Tree Boosting** is difficult to scale.  So we'll also look into **Stochastic Gradient Descent** (SGD) that is a more efficient and faster model with great predictive power.  SGD evaluates and updates the coefficients of the model  at each iteration of minimizing prediction errors on the training data.  The model makes prediction on each training instance, then computes the prediction errors, and finally updates the coefficients in order to decrease the prediction error for the next instance.  Even though SGD is a faster model for large data sets, its disadvantages include needing to tune hyperparameters like regularization and iterations and being sensitive to feature scaling.  SDG is also a good candidate for this problem due to its speed and predictive power.

### Setup
The following codes initialize three helper functions for training and testing the three supervised learning models I've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [110]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, I will now import the three supervised learning models of my choice and run the `train_predict` function for each one. I will train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, I should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, I will implement the following:
- Import the three supervised learning models I've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model I use, if provided.
 - **Note:** I'll use the default settings for each model — I will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [135]:
#Importing the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

#Initializing the three models
clf_A = DecisionTreeClassifier(random_state = 42)
clf_B = GradientBoostingClassifier(random_state = 42)
clf_C = SGDClassifier(random_state = 42)
clf_list = [clf_A, clf_B, clf_C]

#Setting up the training set sizes
X_train_100 = X_train.head(n=100)
y_train_100 = y_train.head(n=100)

X_train_200 =  X_train.head(n=200)
y_train_200 =  y_train.head(n=200)

X_train_300 = X_train
y_train_300 = y_train

#Executing the 'train_predict' function for each classifier and each training set size
for i in clf_list:
    train_predict(i, X_train_100, y_train_100, X_test, y_test)
    print "--------------------------------------------------"
    train_predict(i, X_train_200, y_train_200, X_test, y_test)
    print "--------------------------------------------------"
    train_predict(i, X_train_300, y_train_300, X_test, y_test)
    print "--------------------------------------------------"
    

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6452.
--------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7258.
--------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0030 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6838.
--------------------------------------------------
Training a GradientBoostingClassifier using a training set size of 100. . .
Trained model in 0.0940 seconds
Made predicti

### Tabular Results

** Training and testing times are in seconds **

** Classifer 1 Decision Tree **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0020                  | 0.0010                  |  1.0             |      0.6452     |
| 200               | 0.0040                  | 0.0010                 |  1.0             |       0.7258   |
| 300               | 0.0080                  | 0.0000                 |  1.0             |       0.6838    |

** Classifer 2 - Gradient Tree Boosting**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0730                 |      0.0010            |   1.0000         |  0.7500         |
| 200               |  0.1040                 |      0.0010            |   0.9924         |  0.7313         |
| 300               |  0.1140                 |      0.0010            |   0.9781         |  0.7463         |

** Classifer 3 - SGD**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0010                 |    0.0000              | 0.8075           | 0.8025          |
| 200               |  0.0020                 |    0.0010              | 0.8212           |  0.7703         |
| 300               |  0.0020                 |    0.0000              | 0.7960           |  0.7500         |

## Choosing the Best Model
In this final section, I will choose from the three supervised learning models the *best* model to use on the student data. I will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### 3 - Choosing the Best Model

Precision is the probability that the model's prediction is correct, while recall indicates the percentage of instances predicted correctly out of all the total true instances. F1 score is then the weighted average of precision and recall.  SGD is the most appropriate model for this problem because it has the fastest model training and testing time and the highest training and testing F1 scores across all training and test sizes.  In other words, SGD runs fast and predicts better than the other two models,and its similar F1 scores for both training and testing sets show that the model doesn't overfit as much.  Therefore, a fast model with predictive power will help school personnel devise strategies to improve student retention in a timely and effectively manner.  

### 4 - Model in Layman's Terms

**Stochastic Gradient Descent** (SGD) is a great candidate for predicting student retention because of it's an efficient and fast model with great predictive power.  Using linear SVM classifier, SDC evaluates and updates features' importance or weights contributing to student's success or failure at each iteration of minimizing prediction errors on the training data. SVM classifies a student's success or failure by using a hyperplane of features that separate instances of success and failure as best as possible. The model makes prediction on each training instance, then computes the prediction errors, and finally updates the features' importance in order to decrease the prediction error for the next instance.  After getting the best parameters that give the best predictions on the training data, SGD then uses these parameters to make the final predictions on the testing data.  Compared to Decision Tree and Gradient Tree Boosting, SGD makes predictions on training and testing data more accurately and quickly.  We could then use the feature weights in SVM as a ranking metric for determining each feature's relevance to a student's success or failure.

### Implementation: Model Tuning
I will tune the SDG model using grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. I will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the chosen classifier and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [136]:
#Importing 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV,ShuffleSplit
from sklearn.metrics import make_scorer, f1_score

#Rescaling data

#Creating a parameter list I want to tune: tuning regularization terms and epochs 
parameters = {'alpha': [0.00101, 0.001, 0.01, 0.1, 1, 10, 100, 1000], #regularization terms
              'n_iter': [5,50, 100,500,1000], #epochs
             }

#Create cross-validation sets from the training data
cv_sets = ShuffleSplit(n_splits=10, test_size=0.2, train_size=0.8, random_state=42)

#Initializing the classifier
clf = SGDClassifier(random_state = 42)

#Making an f1 scoring function using 'make_scorer' 
def f1(y_true, y_predict):
    score = f1_score(y_true, y_predict,average='weighted')
    return score
    
f1_scorer = make_scorer(f1,greater_is_better=True)

#Performing grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, cv=cv_sets,scoring=f1_scorer)

#Fitting the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

#Getting the estimator
clf = grid_obj.best_estimator_

#Reporting the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0010 seconds.
Tuned model has a training F1 score of 0.8444.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.7606.


### 5 - Final F<sub>1</sub> Score

The final F1 scores for training and testing data are 0.8444 and 0.7606, respectively.  Compared to the untune model, the tuned model gives us these final F1 scores with ~5% more accurate predictions on the training data and ~1% more accurate predictions on the testing data while still having relatively similar run-times of ~0.000 seconds. In order to better improve the F1 scores in the future, we may need to scale the data, perform feature engineering, and/or tune more parameters.   

## Sources


http://www.saedsayad.com/decision_tree.htm  
http://www.simafore.com/blog/bid/62333/4-key-advantages-of-using-decision-trees-for-predictive-analytics  
https://www.analyticsvidhya.com/blog/2015/11/quick-introduction-boosting-algorithms-machine-learning/  
https://machinelearningmastery.com/gradient-descent-for-machine-learning/  
https://machinelearningmastery.com/implement-linear-regression-stochastic-gradient-descent-scratch-python/  
http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.70.9598&rep=rep1&type=pdf
http://scikit-learn.org/stable/